# Precios para crédito espejo

Acá va una iniciativa de consolidación a clientes y división de opciones de qué cliente abordar por los meses que faltan en el año.

In [21]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from utils import ingresos_financieros, reservas
import numpy as np


In [2]:
datos = pd.read_csv(r"C:\Users\mariajose_chinchilla\Desktop\datos comunes\cosecha 2024.csv", sep=";")
deuda_externa = pd.read_csv(r"C:\Users\mariajose_chinchilla\Documents\GitHub\Modelo_precificacion_creditos\db\arch\info_externa.csv", sep=";")
deuda_externa = deuda_externa[(deuda_externa["desctipogara"] == "Fiduciaria") & (deuda_externa["desctipoactivo"] == "Prestamos")]

In [3]:
# pegar informacion de deuda externa
datos = pd.merge(datos, deuda_externa, how="left", left_on="dpi", right_on="CUI")
datos["PORCENTAJE_SALDO"] = 1 - datos["SALDO"] / datos["MONTO_DESEMBOLSADO"]

In [4]:
# sumar saldo con deuda externa
datos["SALDO TOTAL FIDUCIARIO"] = datos["SALDO"] + datos["saldo"]
datos = datos[~(datos["SALDO TOTAL FIDUCIARIO"].isna()) & (datos["PORCENTAJE_SALDO"] < 0.15)]
# pegar ingresos financieros a varios meses para comparar luego
for i in range(1,7):
    try:
        datos[f"INGRESOS A {i} MESES"] = datos.apply(lambda x: ingresos_financieros(x["SALDO TOTAL FIDUCIARIO"],x["TASA_ORIGINAL"]/1200, x["PLAZO_ORIGINAL"]/30, i), axis=1)
    except Exception as e:
        print(e)

# calcular reservas para monto total fiduciario 
""" Algo importante a tomar en cuenta es que se está tomando una fotografía estática del saldo fiduciario, 
pero se podría actualizar sabiendo la tasa y plazos externos para hacer el cambio de capital externo
y actualizar en el saldo interno para ver si es posible consolidar con la evolución en el tiempo."""

datos["RESERVAS RECREDITO"] = reservas(True) * datos["SALDO TOTAL FIDUCIARIO"]
datos["RESERVAS CREDITO NUEVO"] = reservas(False) * datos["SALDO TOTAL FIDUCIARIO"]

In [5]:
# filtrar datos que sirven únicamente 
datos_temp = datos[["FECHA_APERTURA", "FEC_SALDO", "NO_CREDITO", "CODIGO_CLIENTE", "TIPO_CLIENTE", 
                    "scoreDeudorInstallmentQualPricing",
                    "TASA_ORIGINAL", "PLAZO_ORIGINAL", "DESEMBOLSO", "MONTO_DESEMBOLSADO",
                    "ES_RECREDITO", "PORCENTAJE_SALDO", "SALDO TOTAL FIDUCIARIO", 
                    "INGRESOS A 1 MESES", "INGRESOS A 2 MESES", "INGRESOS A 3 MESES", 
                    "INGRESOS A 4 MESES", "INGRESOS A 5 MESES", "INGRESOS A 6 MESES",
                    "RESERVAS RECREDITO", "RESERVAS CREDITO NUEVO"]]

datos_temp.rename(columns={"scoreDeudorInstallmentQualPricing": "Score"}, inplace=True)

## Leer esquema de precios propuestos

In [6]:
precios = pd.read_excel(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\db\Precios crédito espejo.xlsx")

In [7]:
# buscar la tasa propuesta para colocarle el precio al crédito nuevo
for no_cred, saldo, score in zip(datos_temp["NO_CREDITO"], datos_temp["SALDO TOTAL FIDUCIARIO"], datos_temp["Score"]):
    try:
        precio = precios.loc[(precios["Min"] <= saldo) & (precios["Max"] >= saldo), score].iloc[0]
        datos_temp.loc[datos_temp["NO_CREDITO"] == no_cred,"TASA CRED NUEVO"] = precio
    except:
        if saldo > 500000:
            datos_temp.loc[datos_temp["NO_CREDITO"] == no_cred,"TASA CRED NUEVO"] = -1

In [12]:
# Calcular ingresos desde 1 a 6 meses con la tasa del espejo
for i in range(1,7):
    try:
        datos_temp[f"INGRESOS A {i} MESES CRED ESPEJO"] = datos_temp.apply(lambda x: ingresos_financieros(x["SALDO TOTAL FIDUCIARIO"],x["TASA CRED NUEVO"]/1200, x["PLAZO_ORIGINAL"]/30, i), axis=1)
    except Exception as e:
        print(e)

In [26]:
for i in range(1, 7):
    try:
        # Crear la columna para cada mes
        col_ingresos = f"INGRESOS A {i} MESES"
        col_cred_espejo = f"RECRED EN MES {i}"

        condicion = datos_temp[col_ingresos] >= datos_temp["RESERVAS RECREDITO"]

        datos_temp[col_cred_espejo] = np.where(condicion, 1, 0)

    except Exception as e:
        datos_temp[col_cred_espejo] = 0


In [27]:
# ver si se puede pagar reestructura
for i in range(1, 7):
    try:
        # Crear la columna para cada mes
        col_ingresos = f"INGRESOS A {i} MESES CRED ESPEJO"
        col_cred_espejo = f"CRED ESPEJO EN MES {i}"

        condicion = datos_temp[col_ingresos] >= datos_temp["RESERVAS CREDITO NUEVO"]

        datos_temp[col_cred_espejo] = np.where(condicion, 1, 0)

    except Exception as e:
        datos_temp[col_cred_espejo] = 0


In [29]:
# exportar datos
exportacion = datos_temp[["NO_CREDITO", "CODIGO_CLIENTE", "Score", "TASA_ORIGINAL", "PLAZO_ORIGINAL", "MONTO_DESEMBOLSADO", 'PORCENTAJE_SALDO',
       'SALDO TOTAL FIDUCIARIO', 'RESERVAS RECREDITO', 'RESERVAS CREDITO NUEVO', 'TASA CRED NUEVO',
       'CRED ESPEJO EN MES 1',
       'CRED ESPEJO EN MES 2', 'CRED ESPEJO EN MES 3', 'CRED ESPEJO EN MES 4',
       'CRED ESPEJO EN MES 5', 'CRED ESPEJO EN MES 6', 'RECRED EN MES 1',
       'RECRED EN MES 2', 'RECRED EN MES 3', 'RECRED EN MES 4',
       'RECRED EN MES 5', 'RECRED EN MES 6']]